# Decision Tree Classification

In [1]:
from sklearn.datasets import  load_iris

In [2]:
iris = load_iris()
data =  iris.data
target = iris.target

In [3]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3)

In [5]:
from sklearn.tree import DecisionTreeClassifier

In [6]:
model = DecisionTreeClassifier()

In [11]:
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [14]:
predictions = model.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score

In [19]:
accuracy = accuracy_score(y_test, predictions)

In [21]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.8888888888888888


# Hyperparameter Tuning using GridSearchCV

In [25]:
from sklearn.model_selection import GridSearchCV

In [27]:
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [29]:
grid_search = GridSearchCV(model, param_grid, cv=3)

In [31]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [3, 5, 10, None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]})

In [33]:
grid_search.best_params_

{'criterion': 'gini',
 'max_depth': 3,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [35]:
best_model = grid_search.best_estimator_

In [37]:
best_model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3)

In [39]:
predictions = best_model.predict(X_test)

In [41]:
accuracy = accuracy_score(y_test, predictions)

In [43]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.8888888888888888


# Hyperparameter Tuning using Optuna

In [46]:
import optuna

In [48]:
# Define objective function for Optuna
def objective(trial):
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss'])
    max_depth = trial.suggest_int('max_depth', 3, 20, step=1)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20, step=1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10, step=1)

    model = DecisionTreeClassifier(
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    Accuracy = accuracy_score(y_test, y_pred)
    return Accuracy

In [52]:
# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-03-01 13:59:44,053] A new study created in memory with name: no-name-f681b1f6-ed38-40ae-84ea-4cdd52e68a8d
[I 2025-03-01 13:59:44,061] Trial 0 finished with value: 0.8888888888888888 and parameters: {'criterion': 'entropy', 'max_depth': 19, 'min_samples_split': 15, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.8888888888888888.
[I 2025-03-01 13:59:44,067] Trial 1 finished with value: 0.8888888888888888 and parameters: {'criterion': 'log_loss', 'max_depth': 11, 'min_samples_split': 17, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.8888888888888888.
[I 2025-03-01 13:59:44,071] Trial 2 finished with value: 0.8888888888888888 and parameters: {'criterion': 'log_loss', 'max_depth': 7, 'min_samples_split': 4, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.8888888888888888.
[I 2025-03-01 13:59:44,076] Trial 3 finished with value: 0.8888888888888888 and parameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 17, 'min_samples_leaf': 1}. Best is tr

In [54]:
study.best_params

{'criterion': 'entropy',
 'max_depth': 19,
 'min_samples_split': 15,
 'min_samples_leaf': 6}

In [56]:
study.best_value

0.8888888888888888

In [58]:
best_params = study.best_params

In [70]:
best_model1 = DecisionTreeClassifier(**best_params)

In [72]:
best_model1.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=19, min_samples_leaf=6,
                       min_samples_split=15)

In [74]:
predictions = best_model1.predict(X_test)

In [76]:
accuracy = accuracy_score(y_test, predictions)

In [78]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.8888888888888888
